<a href="https://colab.research.google.com/github/crystaltan1128/lab_02/blob/main/GeotaggedTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets-2.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "my_consumer_key_removed"
    consumer_secret = "my_consumer_secret_removed"
    access_token = "my_access_token_removed"
    access_token_secret = "my_access_token_secret_removed"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-29.23029213413247, 36.680521567440124, 31.150561500656654, 70.61762237231761]  # Europe (this is the second run of the code)

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1518426422003568641, 'username': 'matrix_model', 'created_at': 'Mon Apr 25 03:07:24 +0000 2022', 'lng': 6.1430255, 'lat': 46.204758, 'text': '#OnThisDay University of Helsinki, 2015-department of clinical pharmacology / yes I had pink hair https://t.co/0U0QMKDcUf'}
{'id': 1518426456665247745, 'username': 'cndiablo222', 'created_at': 'Mon Apr 25 03:07:32 +0000 2022', 'lng': -1.014897, 'lat': 51.449484, 'text': 'God bless #Sunny #Chander #Isaak from #NHS they are my best friends now all I care about is recovering after… https://t.co/sXLLb9M5b1'}
{'id': 1518426461610389504, 'username': 'timmsy_ks', 'created_at': 'Mon Apr 25 03:07:34 +0000 2022', 'lng': -2.096208, 'lat': 53.6011235, 'text': "You can't post this without some crackhead chirping in that if you book four years in advance for the 0615 on a Sun… https://t.co/yequWId5Qo"}
{'id': 1518426500575510529, 'username': 'evgenidilov', 'created_at': 'Mon Apr 25 03:07:43 +0000 2022', 'lng': -4.2417514999999995, 'lat': 55.858302499999

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
